# Building LLM applications: Notebook 01

# Langchain basics

## Initialize

In [14]:
import os
import dotenv

from langchain_ollama import ChatOllama
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate

In [15]:
MODEL = 'llama3.2'

In [16]:
# Read fro `.env` file
dotenv.load_dotenv()

OLLAMA_URL = os.getenv('OLLAMA_URL')
print(f"Using Ollama server: {OLLAMA_URL if OLLAMA_URL else 'local'}")

Using Ollama server: local


## Excercise 01: Connect to an LLM

Connect to an LLM and make a simple request (e..g translate a senetence)

```
Translate to French: I like coffee.
```

Take a look at the LLM parameters, and full response 

In [ ]:
# Create the LLM object
llm = ChatOllama(model=MODEL, base_url=OLLAMA_URL)

messages = "Translate to French: I like coffee."

# Invoke the LLM
ret = llm.invoke(messages)
print(ret.content)

In [ ]:
# Show details of the response
print(ret.model_dump_json(indent=2))

## Excercise 02: Prompt template

In [ ]:
# Create the LLM object
llm = ChatOllama(model=MODEL, base_url=OLLAMA_URL)

# Create a prompt from a template
prompt_template = """
Translate to French:

{sentence}

"""

# Create a prompt object from the string template
prompt = PromptTemplate.from_template(prompt_template)


# Create a "chain" of runnable objects
runnable = prompt | llm

# Invoke the 'runnable'
ret = runnable.invoke({"sentence": "I like coffee."})
print(ret.content)

## Excercise 03: Streaming

In [ ]:
# Create the LLM object
llm = ChatOllama(model=MODEL, base_url=OLLAMA_URL)

# Create a prompt from a template
prompt_template = """
Translate to French:

{sentence}

"""

# Create a prompt object from the string template
prompt = PromptTemplate.from_template(prompt_template)


# Create a "chain" of runnable objects
runnable = prompt | llm

# Invoke the 'runnable' as a stream, print each block as it arrives
blocks = []
for b in runnable.stream({"sentence": "I like coffee."}):
    print(b.content, end='')
    blocks.append(b)

print(f"\n----\nReceived {len(blocks)} blocks")

In [ ]:
# What does a block look like
print(blocks[0])

## Excercise 04: Roles

In [ ]:
# Create the LLM object
MODEL = 'llama3.2:3b-instruct-fp16'
llm = ChatOllama(model=MODEL, base_url=OLLAMA_URL)

messages = [
    ('system', "You are helping to translate the user's inputs from english to french"),
    ('human', "I like coffee")
]

ret = llm.invoke(messages)
print(ret.content)

## Excercise 04: Message prompt templates with roles

In [ ]:
# Create the LLM object
MODEL = 'llama3.2:3b-instruct-fp16'


messages_template = [
    ('system', "You are helping to translate the user's inputs from English to {language}"),
    ('human', "{sentence}")
]

llm = ChatOllama(model=MODEL, base_url=OLLAMA_URL)
prompt = ChatPromptTemplate.from_messages(messages=messages_template)
runnable = prompt | llm

# Run the 'runnable'
ret = runnable.invoke({"language": "French", "sentence": "I like coffee."})
print(ret.content)